In [2]:
import tensorflow as tf
import time
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import warnings
warnings.filterwarnings("ignore")

In [3]:
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)
X_train, X_test, y_train, y_test  = mnist.train.images , mnist.test.images, mnist.train.labels, mnist.test.labels

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [6]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((55000, 28, 28, 1), (10000, 28, 28, 1), (55000, 10), (10000, 10))

In [5]:
# tensorflow neural nets take input of image in (batch, width, height, channels)
# Changing dimensions
X_train = X_train.reshape(-1,28,28,1) # image,width,height,channel
X_test  = X_test.reshape(-1,28,28,1)

In [12]:
def model(X,w1,w2,w3,w4,w0,p_keep_conv,p_keep_hidden):
    
    l1_activation = tf.nn.relu(tf.nn.conv2d(X,w1,strides=[1,1,1,1], padding='SAME'))                   # (?,28,28,32)
    l1_           = tf.nn.max_pool(l1_activation,ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')  # (?,14,14,32)
    l1_           = tf.nn.dropout(l1_, p_keep_conv) 
    
    l2_activation = tf.nn.relu(tf.nn.conv2d(l1_,w2,strides=[1,1,1,1], padding='SAME'))                   # (?,14,14,64)
    l2_           = tf.nn.max_pool(l2_activation ,ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')    # (?,7,7,64)
    l2_           = tf.nn.dropout(l2_, p_keep_conv) 
    
    l3_activation = tf.nn.relu(tf.nn.conv2d(l2_,w3,strides=[1,1,1,1], padding='SAME'))                # (?,7,7,128)
    l3_           = tf.nn.max_pool(l3_activation,ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')  # (?,4,4,128)
    l3_           = tf.reshape(l3_, [-1,w4.get_shape().as_list()[0]])                                 # (?, 2048)
    l3_           = tf.nn.dropout(l3_, p_keep_conv) 
    
    l4_ = tf.nn.relu(tf.matmul(l3_,w4))
    l4_ = tf.nn.dropout(l4_, p_keep_hidden) 
    
    px = tf.matmul(l4_, w0)
    return px


In [13]:
tf.reset_default_graph()

# inputs
X = tf.placeholder(name='Images', dtype=tf.float32, shape=[None,28,28,1])
y = tf.placeholder(name='y_true', dtype=tf.float32, shape=[None,10])

# weights
w1 = tf.get_variable(name='w_1',shape=[3,3, 1, 32],initializer=tf.random_normal_initializer())
w2 = tf.get_variable(name='w_2',shape=[3,3,32, 64],initializer=tf.random_normal_initializer())
w3 = tf.get_variable(name='w_3',shape=[3,3,64,128],initializer=tf.random_normal_initializer())
w4 = tf.get_variable(name='w_4',shape=[128*4*4,625],initializer=tf.random_normal_initializer())
w0 = tf.get_variable(name='w_0',shape=[625, 10],initializer=tf.random_normal_initializer())

# probability for keep dropout
p_keep_conv   = tf.placeholder(tf.float32)
p_keep_hidden = tf.placeholder(tf.float32)

# model structure
mdl = model(X,w1,w2,w3,w4,w0,p_keep_conv,p_keep_hidden)

# cost function evaluation
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=mdl,labels=y))

# training operations
train_op   = tf.train.RMSPropOptimizer(0.001).minimize(cost) # Back propagation
predict_op = tf.argmax(mdl,1)                                # Forward propagation


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [14]:
epoch = 2
batch_size = 128
ss =  tf.Session() 
ss.run(tf.global_variables_initializer())
for i in range(epoch): 
        for start, end in zip(range(0, len(X_train), batch_size), range(128, len(X_train)+1, batch_size)):
        
            ss.run(train_op, feed_dict={X: X_train[start:end], 
                                        y: y_train[start:end],
                                        p_keep_conv:0.8,
                                        p_keep_hidden:0.6
                                       })
        print('epoch:{} cost: {} estimate_time: {}'.format(i,ss.run(cost,feed_dict={X: X_train[start:end], 
                                                                  y: y_train[start:end],
                                                                  p_keep_conv:0.8,
                                                                  p_keep_hidden:0.6})
                                                          ))
y_pred = ss.run(predict_op,feed_dict={X:X_test,
                                          p_keep_conv:1,
                                          p_keep_hidden:1})

epoch:0 cost: 9370.302734375 estimate_time: 0.8922717571258545
epoch:1 cost: 1397.720947265625 estimate_time: 0.42505621910095215


In [15]:
c = 0
for i,j in enumerate(y_pred):
    if np.argmax(y_test,1)[i] == j:
        c+=1
print("accuracy : ",(c/len(y_pred))*100)

# Saving for tensorboard
tf.summary.FileWriter('./logs',ss.graph)

accuracy :  64.11
